In [1]:
import json

qas = json.load(open("./data/qasper_reranked_t53b.json",'r'))

In [ ]:
import json

data = json.load(open("data/qasper/qasper-test-v0.3.json","r"))
questions = {}
for k in data:
    for qa in data[k]['qas']:
        questions[qa['question_id']] = qa
    

In [ ]:
## use this to select a stratified subset of the datase.
extractives = []
abstractives = []
none_ = []
yes_no = []
stratfy_by = []
for item in qas:
    annotation_info = questions[item['question_id']]["answers"][0]
    # annotation_info = item['answers'][0]
    answer_info = annotation_info["answer"]
    if answer_info["unanswerable"]:
        stratfy_by.append(0)
        none_.append(item)
    else:
        if answer_info["extractive_spans"]:
            extractives.append(item)
            stratfy_by.append(1)
        elif answer_info["free_form_answer"]:
            # answer = answer_info["free_form_answer"]
            abstractives.append(item)
            stratfy_by.append(2)
            # answer_type = "abstractive"
        elif answer_info["yes_no"]:
            yes_no.append(item)
            stratfy_by.append(3)
        elif answer_info["yes_no"] is not None:
            yes_no.append(item)
            stratfy_by.append(3)

len(extractives), len(abstractives),len(none_), len(yes_no)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, test, y_train, y_test = train_test_split(qas, stratfy_by, test_size=0.15, random_state=42, stratify=stratfy_by)
len(test)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5')

In [ ]:
train = json.load(open("data/qasper/explained_train.json",'r'))
train_embeddings = model.encode([a['question'] for a in train])

In [ ]:
import os
import openai

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

openai.api_key = os.getenv("OPENAI_API_KEY")

def generate(prompt,max_tokens=1000, temperature=0):
    # tokens = tokenizer.tokenize(prompt)
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=temperature,
        max_tokens=256,
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response["choices"][0]['text']
# prompt="Classify the documents as relevant or irrelevant to answer the question. Explain the decisions. Then, answer the question using the relevant documents.\n\nExample 1:\n\n[Document 1]: In this section we describe a number of experiments targeted to compare the performance of popular named entity recognition algorithms on our data. We trained and evaluated Stanford NER, spaCy 2.0, and a recurrent model similar to BIBREF13 , BIBREF14 that uses bidirectional LSTM cells for character-based feature extraction and CRF, described in Guillaume Genthial's Sequence Tagging with Tensorflow blog post BIBREF15 .\n[Document 2]: Stanford NER is conditional random fields (CRF) classifier based on lexical and contextual features such as the current word, character-level n-grams of up to length 6 at its beginning and the end, previous and next words, word shape and sequence features BIBREF16 .\n[Document 3]: spaCy 2.0 uses a CNN-based transition system for named entity recognition. For each token, a Bloom embedding is calculated based on its lowercase form, prefix, suffix and shape, then using residual CNNs, a contextual representation of that token is extracted that potentially draws information from up to 4 tokens from each side BIBREF17 . Each update of the transition system's configuration is a classification task that uses the contextual representation of the top token on the stack, preceding and succeeding tokens, first two tokens of the buffer, and their leftmost, second leftmost, rightmost, second rightmost children. The valid transition with the highest score is applied to the system. This approach reportedly performs within 1% of the current state-of-the-art for English . In our experiments, we tried out 50-, 100-, 200- and 300-dimensional pre-trained GloVe embeddings. Due to time constraints, we did not tune the rest of hyperparameters and used their default values.\n[Document 4]: In order to evaluate the models trained on generated data, we manually annotated a named entities dataset comprising 53453 tokens and 2566 sentences selected from over 250 news texts from ilur.am. This dataset is comparable in size with the test sets of other languages (Table TABREF10 ). Included sentences are from political, sports, local and world news (Figures FIGREF8 , FIGREF9 ), covering the period between August 2012 and July 2018. The dataset provides annotations for 3 popular named entity classes: people (PER), organizations (ORG), and locations (LOC), and is released in CoNLL03 format with IOB tagging scheme. Tokens and sentences were segmented according to the UD standards for the Armenian language BIBREF11 .\n[Document 5]: The main model that we focused on was the recurrent model with a CRF top layer, and the above-mentioned methods served mostly as baselines. The distinctive feature of this approach is the way contextual word embeddings are formed. For each token separately, to capture its word shape features, character-based representation is extracted using a bidirectional LSTM BIBREF18 . This representation gets concatenated with a distributional word vector such as GloVe, forming an intermediate word embedding. Using another bidirectional LSTM cell on these intermediate word embeddings, the contextual representation of tokens is obtained (Figure FIGREF17 ). Finally, a CRF layer labels the sequence of these contextual representations. In our experiments, we used Guillaume Genthial's implementation of the algorithm. We set the size of character-based biLSTM to 100 and the size of second biLSTM network to 300\n\nQuestion: what ner models were evaluated?\n\nRelevant documents:\n[Document 1] is relevant to answer the question because it describes a number of experiments that compare the performance of popular named entity recognition algorithms.\n[Document 2] is relevant to answer the question because it describes the Stanford NER algorithm, which is one of the models that was evaluated.\n[Document 3] is relevant to answer the question because it describes the spaCy 2.0 algorithm, which is one of the models that was evaluated.\n[Document 5] is relevant to answer the question because it describes the recurrent model with a CRF top layer, which is one of the models that was evaluated.\n\nIrrelevant documents:\n[Document 4] is not relevant to answer the question because it describes a named entities dataset that was used to evaluate the models, but it does not describe any of the models themselves.\n\nAnswer: Stanford NER algorithm, the spaCy 2.0 algorithm, recurrent model with a CRF top layer.\n\nExample 2:\n\n[Document 1]: Social media are becoming an increasingly important source of information about the public mood regarding issues such as elections, Brexit, stock market, etc. In this paper we focus on sentiment classification of Twitter data. Construction of sentiment classifiers is a standard text mining task, but here we address the question of how to properly evaluate them as there is no settled way to do so. Sentiment classes are ordered and unbalanced, and Twitter produces a stream of time-ordered data. The problem we address concerns the procedures used to obtain reliable estimates of performance measures, and whether the temporal ordering of the training and test data matters. We collected a large set of 1.5 million tweets in 13 European languages. We created 138 sentiment models and out-of-sample datasets, which are used as a gold standard for evaluations. The corresponding 138 in-sample datasets are used to empirically compare six different estimation procedures: three variants of cross-validation, and three variants of sequential validation (where test set always follows the training set). We find no significant difference between the best cross-validation and sequential validation. However, we observe that all cross-validation variants tend to overestimate the performance, while the sequential methods tend to underestimate it. Standard cross-validation with random selection of examples is significantly worse than the blocked cross-validation, and should not be used to evaluate classifiers in time-ordered data scenarios.\n[Document 2]: The complexity of Twitter data raises some challenges on how to perform such estimations, as, to the best of our knowledge, there is currently no settled approach to this. Sentiment classes are typically ordered and unbalanced, and the data itself is time-ordered. Taking these properties into account is important for the selection of appropriate estimation procedures.\n[Document 3]: We collected a large corpus of nearly 1.5 million Twitter posts written in 13 European languages. This is, to the best of our knowledge, by far the largest set of sentiment labeled tweets publicly available. We engaged native speakers to label the tweets based on the sentiment expressed in them. The sentiment label has three possible values: negative, neutral or positive. It turned out that the human annotators perceived the values as ordered. The quality of annotations varies though, and is estimated from the self- and inter-annotator agreements. All the details about the datasets, the annotator agreements, and the ordering of sentiment values are in our previous study BIBREF22 . The sentiment distribution and quality of individual language datasets is in Table TABREF2 . The tweets in the datasets are ordered by tweet ids, which corresponds to ordering by the time of posting.\n\nQuestion: In what way are sentiment classes ordered?\n\nRelevant documents:"

# generate(prompt)

In [ ]:
import random

kshot = 3
RANDOM = True
STATIC = False

if STATIC:
    static_sample = random.sample(train,k=kshot)
def generate_explanation_answer(question, documents):
    prompt="For each example, use the documents to create an \"Answer\" and an \"Explanation\" to the \"Question\". Answer \"Unanswerable\" when not enough information is provided in the documents. Pay attention to answer only \"yes\" or \"no\" in boolean questions.\n\n"
    if RANDOM:
        hits = random.sample(train,k=kshot)
    elif STATIC:
        hits = static_sample
    else:
        item_embeddings = model.encode(question)

        all_top = util.dot_score(item_embeddings, train_embeddings)[0].topk(kshot)

        hits = np.array(train)[all_top.indices].tolist()

        hits.reverse()

    for i, hit in enumerate(hits):
        prompt += "Example {0}:\n\n".format(i+1)

        for j,d in enumerate(hit['evidence']):
            prompt += "[Document {0}]: {1}\n\n".format(j+1, d)    

        prompt += "Question: {0}\n\nExplanation: {1}\n\nAnswer: {2}\n\n".format(hit['question'],hit['explanation'],hit['answer'])

    prompt += "Example {0}:\n\n".format(i+2)


    for k, document in enumerate(documents):
        prompt += "[Document {0}]: {1}\n\n".format(k+1, document['title'])
    
    prompt+= "Question: {0}\n\nExplanation:".format(question)

    tokens = tokenizer.tokenize(prompt)
    if len(tokens) >= 3800:
        return generate_explanation_answer(question, documents)
    
    res = generate(prompt)

    if "answer:" not in res.lower():
        prompt = prompt+res+"\n\nAnswer:"
        res = res+"\n\nAnswer:"+ generate(prompt)
    
    explanation = res.lower().split("answer:")[0]
    
    answer = res.lower().split("answer:")[1]

    return explanation, answer, prompt, res
    


In [ ]:
import re
from tqdm import tqdm
import numpy as np
from datetime import date
import random 

RANDOM = True

kshot = 3
docs = 3

regex = r"\[document \d+\]"


f = open("./results/qasper.jsonl",'w')
# results =
logs = []
tokens= []
for item in tqdm(test):

    documents = item['documents'][:docs]
    explanation, answer, prompt, res = generate_explanation_answer(item['question'], documents)

    matches = re.finditer(regex, explanation, re.MULTILINE)
    relevant_documents = []
    for match in matches:
        nums = re.findall(r'\b\d+\b', match.group())
        if len(nums) > 0:
            try:
                relevant_documents.append(documents[int(nums[0])-1]['title'])
            except:
                pass

    f.write(json.dumps({
        'question_id': item['question_id'], 
        'predicted_answer': answer.rstrip(), 
        'predicted_evidence': relevant_documents
    })+"\n")

    logs.append({
        'question_id': item['question_id'],
        'predicted_answer': answer.rstrip(),
        'predicted_evidence': relevant_documents,
        'prompt': prompt,
        "res": res
    })  

## Evaluation

In [ ]:
!python3 qasper_evaluator.py --predictions results/qasper.jsonl --gold data/qasper-test-v0.3.json --text_evidence_only